# Dados Abertos - Senado Brasileiro, CEAPS

## Carregando módulos usados na análise.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import re
import csv
import glob
%matplotlib inline

#pd.options.plotting.backend = "plotly"

## Carregando Dados

Diversas inconsistências e dados faltantes em campos.

### Ano de 2022

In [2]:
# header=1 -> ignore first line header
df = pd.read_csv('data/despesa_ceaps_2022.csv', delimiter=';', header=1, encoding='iso-8859-1')
df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,03/01/2022,Despesa com pagamento de aluguel de imóvel par...,6000,2173614
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,04/01/2022,Despesa com divulgação da atividade parlamenta...,1500,2173615
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,14/01/2022,Despesa com produção de texto e edição de víde...,6000,2173616
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,18/01/2022,Divulgação da atividade parlamentar,1000,2173618
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,17/01/2022,Divulgação da atividade parlamentar,2000,2173617


### Outros Anos

In [3]:
datas = {}
for file in sorted(glob.glob('data/*.csv')):
    
    if re.search(r'[0-9]+(.csv)', file):
        print('> load... ', file)
        datas[ re.search(r'[0-9]+', file ).group(0) ] = pd.read_csv(file, delimiter=';', header=1, encoding='iso-8859-1')
#(lambda d, x: [d.pop(i) for i in x] )(datas, ['2008','2009', '2010', '2011','2012'])

> load...  data/despesa_ceaps_2008.csv
> load...  data/despesa_ceaps_2009.csv
> load...  data/despesa_ceaps_2010.csv
> load...  data/despesa_ceaps_2011.csv
> load...  data/despesa_ceaps_2012.csv
> load...  data/despesa_ceaps_2013.csv
> load...  data/despesa_ceaps_2014.csv
> load...  data/despesa_ceaps_2015.csv
> load...  data/despesa_ceaps_2016.csv
> load...  data/despesa_ceaps_2017.csv
> load...  data/despesa_ceaps_2018.csv
> load...  data/despesa_ceaps_2019.csv
> load...  data/despesa_ceaps_2020.csv
> load...  data/despesa_ceaps_2021.csv
> load...  data/despesa_ceaps_2022.csv


In [4]:
dff = pd.concat(datas, ignore_index=True)


In [ ]:
dff.loc[0:dff.shape[0],'VALOR_REEMBOLSADO'].replace(r'(\s)', '', regex=True, inplace=True)
dff.loc[0:dff.shape[0],'VALOR_REEMBOLSADO'].replace(r'(\n)','', regex=True, inplace=True)
dff.loc[0:dff.shape[0], 'VALOR_REEMBOLSADO'].replace(r'(\r)', '', regex=True, inplace=True)
dff.head()

In [ ]:
dff.loc[0:,'VALOR_REEMBOLSADO'].replace( regex=r'(,)', value='.',  inplace=True )
#dff.loc[0:dff.shape[0],'VALOR_REEMBOLSADO'] =dff.loc[0:dff.shape[0],'VALOR_REEMBOLSADO'].replace( regex={r'(,)':'.'} )
#dfs['VALOR_REEMBOLSADO'] = pd.to_numeric(dff['VALOR_REEMBOLSADO'], downcast='float')
dff.head()

In [ ]:
dff.loc[0:dff.shape[0], 'VALOR_REEMBOLSADO'] = pd.to_numeric(dff['VALOR_REEMBOLSADO'], downcast='float')
#dff.dropna(inplace=True)
dff.to_csv('data.csv', index=False)

## Tratamento e Limpeza dos Dados (Data Wrangling)

No campo 'VALOR_REEMBOLSADO' trocar virgula por ponto e converter para tipo numérico (float).

In [ ]:
df['VALOR_REEMBOLSADO'] = pd.to_numeric( df['VALOR_REEMBOLSADO'].replace(regex=r'(,)', value='.'), downcast='float')



In [ ]:
df.head()

Converter campo 'DATA' do tipo string para tipo Date

In [ ]:
# Converter DATA para tipo Date
df['DATA'] = pd.to_datetime(df['DATA'], dayfirst=True)

In [ ]:
df

## Gastos Ano 2022

In [ ]:
senadores = df['SENADOR'].unique()
df_gastos_sn = { 'Senador':[], 'GastoAnual':[], 'RegsSemDocumento':[], 'RegsSemDetalhamentoDoGasto':[] }

In [ ]:
for i in senadores:
    df_gastos_sn['Senador'].append(i)
    df_gastos_sn['GastoAnual'].append( df[ df['SENADOR'] == i ]['VALOR_REEMBOLSADO'].sum() )
    df_gastos_sn['RegsSemDocumento'].append( df[ df['DOCUMENTO'].isna() == True][df['SENADOR'] == i ].isna().sum()['DOCUMENTO'] )
    df_gastos_sn['RegsSemDetalhamentoDoGasto'].append( df[ df['DETALHAMENTO'].isna() == True][df['SENADOR'] == i ].isna().sum()['DETALHAMENTO'] )


In [ ]:
#ds = pd.DataFrame( { 'Senador': df_gastos_sn['Senador'], 'Gastos': df_gastos_sn['GastoAnual'] })
ds = pd.DataFrame( df_gastos_sn )
ds = ds.sort_values(by=['GastoAnual'], ascending=False)
ds

### 10 Senadores com Maior Gastos 2022

In [ ]:
ds[:10].plot(x='Senador', y='GastoAnual', title='Gastos (R$) Senadores Brasileiros 2022', kind='bar')

### 10 Senadores com mais ocorrência de falta de Documento comprobatório do gasto na Base de Dados

In [ ]:
ds.sort_values(by=['RegsSemDocumento'], ascending=False)[:10].plot(x='Senador', y='RegsSemDocumento', title='10 Sen. Sem Documento de Registro do Gasto', kind='bar')

In [ ]:
df[ df['DOCUMENTO'].isna() == True][df['SENADOR'] == 'ZENAIDE MAIA'].isna().sum()